# Chapter 9 - Data Science
## Data Manipulation

## 0 - Setting up the notebook

In [ ]:
import json

import arrow
import numpy as np
import pandas as pd
from pandas import DataFrame

## 1- Loading Data into a DataFrame

In [ ]:
# Load data from a json file into a DataFrame
df = pd.read_json("data.json")

# let's take a peek at the first 5 rows, to make sure
# nothing weird has happened
df.head()

In [ ]:
# OK! DataFrame is alive and well!
# let's get a sense of how many rows there are and
# what is their structure.
df.count()

In [ ]:
df.describe()

In [ ]:
# let's see which are the top 3 campaigns according
# to budget (regardless of the currency)
df.sort_values(by=['cmp_bgt'], ascending=False).head(3)

In [ ]:
# we can also use 'tail' to get the bottom 3 campaigns
df.sort_values(by=['cmp_bgt'], ascending=False).tail(3)

## 2 - Manipulating the DataFrame

In [ ]:
# first, let's explode cmp_name into its components
# and get a separate DataFrame for those

def unpack_campaign_name(name):
    # very optimistic method, assumes data in campaign name
    # is always in good state
    type_, start, end, age, gender, currency = name.split('_')
    start = arrow.get(start, 'YYYYMMDD').date()
    end = arrow.get(end, 'YYYYMMDD').date()
    return type_, start, end, age, gender, currency

campaign_data = df['cmp_name'].apply(unpack_campaign_name)
campaign_cols = [
    'Type', 'Start', 'End', 'Target Age', 'Target Gender',
    'Currency']
campaign_df = DataFrame(
    campaign_data.tolist(), columns=campaign_cols, index=df.index)
campaign_df.head(3)

In [ ]:
# let's join the two dataframes
df = df.join(campaign_df)

In [ ]:
# and take a peek: good! It seems to be ok.
df[['cmp_name'] + campaign_cols].head(3)

In [ ]:
# now let's do the same for the JSON user object

def unpack_user_json(user):
    # very optimistic as well, expects user objects
    # to have all attributes
    user = json.loads(user.strip())
    return [
        user['username'],
        user['email'],
        user['name'],
        user['gender'],
        user['age'],
        user['address'],
    ]

user_data = df['user'].apply(unpack_user_json)
user_cols = [
    'Username', 'Email', 'Name', 'Gender', 'Age', 'Address']
user_df = DataFrame(
    user_data.tolist(), columns=user_cols, index=df.index)

In [ ]:
# let's join the two dataframes
df = df.join(user_df)

In [ ]:
# and take a peek: good! Still in good shape.
df[['user'] + user_cols].head(2)

In [ ]:
# now we have the DataFrame completely expanded, so it's
# time to play with it. First, let's fix those ugly column names
new_column_names = {
    'cmp_bgt': 'Budget',
    'cmp_spent': 'Spent',
    'cmp_clicks': 'Clicks',
    'cmp_impr': 'Impressions',
}
df.rename(columns=new_column_names, inplace=True)

In [ ]:
# let's add three other columns

def calculate_extra_columns(df):
    # Click Through Rate
    df['CTR'] = df['Clicks'] / df['Impressions']
    # Cost Per Click
    df['CPC'] = df['Spent'] / df['Clicks']
    # Cost Per Impression
    df['CPI'] = df['Spent'] / df['Impressions']
    
calculate_extra_columns(df)

In [ ]:
# let's take a peek
df[['Spent', 'Clicks', 'Impressions',
    'CTR', 'CPC', 'CPI']].head(3)

In [ ]:
# let's take the values of the first row and verify
clicks = df['Clicks'][0]
impressions = df['Impressions'][0]
spent = df['Spent'][0]

CTR = df['CTR'][0]
CPC = df['CPC'][0]
CPI = df['CPI'][0]

print('CTR:', CTR, clicks / impressions)
print('CPC:', CPC, spent / clicks)
print('CPI:', CPI, spent / impressions)

In [ ]:
# let's also add the name of the Day when campaign starts
def get_day_of_the_week(day):
    return day.strftime("%A")

def get_duration(row):
    return (row['End'] - row['Start']).days

df['Day of Week'] = df['Start'].apply(get_day_of_the_week)
df['Duration'] = df.apply(get_duration, axis=1)

In [ ]:
# let's verify
df[['Start', 'End', 'Duration', 'Day of Week']].head(3)

In [ ]:
# now let's get rid of the cmp_name and user columns,
# which we don't need any more, and address too
final_columns = [
    'Type', 'Start', 'End', 'Duration', 'Day of Week', 'Budget',
    'Currency', 'Clicks', 'Impressions', 'Spent', 'CTR', 'CPC',
    'CPI', 'Target Age', 'Target Gender', 'Username', 'Email',
    'Name', 'Gender', 'Age'
]
df = df[final_columns]

## 3 - Saving to a file in different formats

In [ ]:
# CSV format
df.to_csv('df.csv')

In [ ]:
# JSON format
df.to_json('df.json')

In [ ]:
# Spreadsheet format
df.to_excel('df.xlsx')

## 4 - Visualizing results

First let's take care of the graphics, we configure the `matplotlib` plot stle and set the font family to `serif`.

In [ ]:
%matplotlib inline

In [ ]:
# make the graphs nicer
import matplotlib.pyplot as plt
plt.style.use(['classic', 'ggplot'])
# see all available with: print(plt.style.available)
plt.rc('font', family='serif')

In [ ]:
df.describe()

In [ ]:
df[['Budget', 'Spent', 'Clicks', 'Impressions']].hist(
    bins=16, figsize=(16, 6));

In [ ]:
df[['CTR', 'CPC', 'CPI']].hist(
    bins=20, figsize=(16, 6));

In [ ]:
# let's see the campaigns whose spent is > than 75% of the budget
selector = (df.Spent > df.Budget * .75)
df[selector][['Budget', 'Spent', 'Clicks', 'Impressions']].hist(
    bins=15, figsize=(16, 6), color='green');

In [ ]:
# Let's aggregate by Day of the Week
df_weekday = df.groupby(['Day of Week']).sum()
df_weekday[['Impressions', 'Spent', 'Clicks']].plot(
    figsize=(16, 6), subplots=True);

In [ ]:
# Let's aggregate by gender and age
agg_config = {
    'Impressions': ['mean', 'std'],
    'Spent': ['mean', 'std'],
}

df.groupby(['Target Gender', 'Target Age']).agg(agg_config)

In [ ]:
# finally, let's make a pivot table
df.pivot_table(
    values=['Impressions', 'Clicks', 'Spent'],
    index=['Target Age'],
    columns=['Target Gender'],
    aggfunc=np.sum
)